<a href="https://colab.research.google.com/github/salexey1990/gb/blob/master/introduction-neural-networks/hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

In [0]:
(train_data, validation_data, test_data), metadata = tfds.load(
    name="imdb_reviews", 
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True)

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [0]:
test_examples_batch, test_labels_batch = next(iter(test_data.batch(10)))
test_examples_batch

In [0]:
train_labels_batch

In [0]:
test_labels_batch

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(32, kernel_initializer='orthogonal', kernel_regularizer=tf.keras.regularizers.l1(0.01), activation='relu'))
model.add(tf.keras.layers.Dense(16, kernel_initializer='orthogonal', kernel_regularizer=tf.keras.regularizers.l1(0.01), activation='relu'))
model.add(tf.keras.layers.Dense(5, kernel_initializer='orthogonal', kernel_regularizer=tf.keras.regularizers.l1(0.01), activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

На этот раз введение регуляризации вместе с увеличением количества эпох дало хорошую прибавку к качестве модели, оцениваемом на тестовых данных. Интересным показалась возможность выбирать параметры матрицы, которой будут инициализироваться веса модели, хоть в данном примере на качестве модели это и не отразилось